In [1]:
############################################################################
### An Agent Based Approach for Understanding Complex Terrorism Behaviors###
############################################################################

#By Salih Tutun, PhD, 

#You can find the paper on https://www.researchgate.net/publication/303692784_An_Agent_Based_Approach_for_Understanding_Complex_Terrorism_Behaviors

from pylab import *
import matplotlib.pyplot as plt
import random
import numpy as np
import random as rd
import networkx as nx
import copy
from random import gauss
from random import randint  
import pylab as pb


Coef_meandefence = 15
min_n = 5
max_n = 10
n = random.randint(min_n,max_n)   # start with random number of attacks
dcooldown=80
World_size_x=World_size_y=50
popsize=50
DNeibor_size=20
defencefactor=7000
aggressivefactor1=10   #15
aggressivefactor2=10
t=0
Kp_mean = 10.49
Wp_mean = 22.87
At = ['Bombing/Explosion','Assassination','Armed Assault','Kidnapping','Barricade Incident','Hijacking','Facility/Infrastructure Attack','Unknown','Unarmed Assault']
Tt = ['Airports','Business','Educational Institution','Government (Diplomatic)','Government (General)','Media','Maritime','Military','NGO','Police','Private Citizens Property','Religion Institutions','Telecommunication','Tourists','Transportation','Unknown','Utilities','Non-state Military','*Unknown']
Wt = ['Chemical','Bombs','Fake Weapons','Firearms','Incendiary','Melee','Unknown','vehicle','*Unknown']
Pp = ['No Damage','Damage','Unknown']
At_p=[.929380,.048748,.008432,.003162,.005534,.002108,.001317,.001054,.000264]   # cumulative distribution fucntion of attributes
Tt_p = [0.007378,0.067984,0.011330,0.020026,0.155204,0.0036890,0.00316206, 0.23083004,0.00237154,0.22108037,0.15177866,0.04453228,0.0002635,0.00342556,0.02898551,0.02766798,0.00131752,0.03188406,0.00711462]
Wt_p = [0.0079051,0.98708827,0.0002635,0.00685112,0.0002635,0.00105402,0.00210804,0.00131752,0.0002635]
Pp_p=[.22108037,.54624506,.23267457]
At_cp = np.cumsum(At_p)
Wt_cp = np.cumsum(Wt_p)
Tt_cp = np.cumsum(Tt_p)
Pp_cp = np.cumsum(Pp_p)
At_size = len(At)
Wt_size = len(Wt)
Tt_size = len(Tt)
Pp_size = len(Pp)
simicons = 0.75
aggressivechange1 = -1
aggressivechange2 = -1
aggressivefade = 0.1
limi = 5
infoshare = 0.67
class people:
    pass

Attri = 6
Weight = 1./Attri

def initialize():
    global g,World_size_x,World_size_y,timet, attnum1,attnum2,t
    global  Defence_level,eventagents,popagents,agfnum1,agfnum2,defnum
    timet=[]
    attnum1=[]
    attnum2=[]
    agfnum1=[]
    agfnum2=[]
    defnum = []
    popagents = []
    for i in range(popsize):
        ag = people()
        ag.x = random.randint(0,World_size_x)
        ag.y = random.randint(0,World_size_y)
        popagents.append(ag)
    
    at1=0
    at2=0
    g = nx.Graph()
    eventagents=[]
    
    
    for i in range(n):
        g.add_node(i)                                   # add initial nodes
        g.node[i]['Terrist Group'] =random.randint(0,2)
        if g.node[i]['Terrist Group'] ==0:
            at1+=1
        else:
            at2+=1
        
        rn=np.random.rand()                        # add attributes based on the distribution
        ind=pb.find(At_cp>=rn)
        g.node[i]['Attack Type']=At[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Wt_cp>=rn)
        g.node[i]['Weapon Type']=Wt[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Tt_cp>=rn)
        g.node[i]['Target Type']=Tt[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Pp_cp>=rn)
        g.node[i]['Property']=Pp[ind[0]]
        

 
        # random choose number of killed and wounded people based on gauss distribution        
        while True:
            Kp = int(gauss(Kp_mean,35.86))
            if Kp >= 0:
                break
            
        while True:
            Wp = int(gauss(Wp_mean,80.44))
            if Wp >= 0:
                break

        g.node[i]['Killed People'] = Kp
        g.node[i]['Wounded People'] = Wp
        
        # calculate the success rate based on the decision tree
        if Kp <= 0:
            Succ = 0
        else:
            if g.node[i]['Attack Type'] != 'Assassination' and g.node[i]['Attack Type'] != 'Facility/Infrastructure Attack':
                Succ = 1
            elif g.node[i]['Attack Type'] == 'Facility/Infrastructure Attack':
                Succ = 0
            else:
                if Kp <=1:
                    Succ = 0
                else:
                    if g.node[i]['Target Type'] != 'Goverment (General)' and g.node[i]['Target Type'] != 'Government (Diplomatic)' and g.node[i]['Target Type'] != 'Private Citizens Property' and g.node[i]['Target Type'] != 'Military' and g.node[i]['Target Type'] != 'Police':
                        Succ = 1
                    elif g.node[i]['Target Type'] == 'Government (Diplomatic)':
                        Succ = 0
                    elif g.node[i]['Target Type'] == 'Goverment (General)':
                        if g.node[i]['Property'] == 'Unknown':
                            Succ = 1
                        else: 
                            Succ = 0
                    elif g.node[i]['Target Type'] == 'Military':
                        if g.node[i]['Property'] != 'Damage':
                            Succ = 1
                        else: 
                            Succ = 0
                    elif g.node[i]['Target Type'] == 'Private Citizens Property':
                        if Wp > 14 and Wp <= 38:
                            Succ = 0
                        else:
                            Succ = 1
                    elif g.node[i]['Target Type'] == 'Police':      
                        if g.node[i]['Property'] == 'Damage':
                            Succ = 1
                        elif g.node[i]['Property'] == 'No Damage':
                            Succ = 0                        
                        else:
                            if Wp > 10:
                                Succ = 0
                            else:
                                Succ = 1
        g.node[i]['Success Rate'] = Succ
        g.node[i]['Location'] = (randint(0,World_size_x),randint(0,World_size_y))   # random location
        
        eventagents.append(g.node[i])
    timet+=[t,]                           # these for drawing 
    attnum1+=[at1,] 
    attnum2+=[at2,]   
    agfnum1+=[aggressivefactor1,]
    agfnum2+=[aggressivefactor2,]
    
    t+=1
    Defence_level=np.zeros([World_size_x,World_size_y]) 
    for i in range(n):
        #g.node[i]['Location'] = (randint(0,Word_size),randint(0,Word_size))
        #Defence_level[g.node[i]['Location'][0],g.node[i]['Location'][1]]=2
        for x in range(World_size_x):
            for y in range(World_size_y):
                if (x-g.node[i]['Location'][0])**2+ (y-g.node[i]['Location'][1])**2<DNeibor_size**2:
                    # calculate the defence level
                    Defence_level[x,y]+=defencefactor*np.exp(-np.sqrt((x-g.node[i]['Location'][0])**2+ (y-g.node[i]['Location'][1])**2))
        
        #  calculate the similarity
    for i in range(n):
        for j in range(i+1,n):
            S_At = 0
            S_Wt = 0
            S_Tt = 0
            S_Kp = 0
            S_Wp = 0
            S_Pp = 0
            if g.node[i]['Attack Type'] == g.node[j]['Attack Type']:
                S_At = 1
            if g.node[i]['Weapon Type'] == g.node[j]['Weapon Type']:
                S_Wt = 1
            if g.node[i]['Target Type'] == g.node[j]['Target Type']:
                S_Tt = 1
            if abs(g.node[i]['Killed People'] - g.node[j]['Killed People']) <= 0.2 * Kp_mean:
                S_Kp = 1
            if abs(g.node[i]['Wounded People'] - g.node[j]['Wounded People']) <= 0.2 * Wp_mean:
                S_Wp = 1
            if g.node[i]['Property'] == g.node[j]['Property']:
                S_Pp = 1
       
            Simi = Weight * S_At + Weight * S_Wt + Weight * S_Tt + Weight * S_Kp + Weight * S_Wp + Weight * S_Pp+np.exp(-np.sqrt((g.node[j]['Location'][0]-g.node[i]['Location'][0])**2+ (g.node[i]['Location'][1]-g.node[i]['Location'][1])**2))*(Defence_level[g.node[i]['Location'][0],g.node[i]['Location'][1]]+Defence_level[g.node[j]['Location'][0],g.node[j]['Location'][1]])
            if Simi >= simicons:
                g.add_edge(i, j)
    
    defnum = [np.mean(Defence_level),]

def observe():
    global g,positions,eventagents,popagents,timet, attnum1,attnum2,t,aggressivefactor1,aggressivefactor2
    global Defence_level,World_size_x,World_size_y,eventsize1, eventsize2,agfnum1,agfnum2,defnum
    
    
    g.pos = nx.spring_layout(g)         # position 
    
    plt.subplot(3, 2, 1)     # network model
    plt.cla()
    nx.draw(g, pos = g.pos, with_labels = True,node_size = [200 if g.node[i]['Success Rate'] == 1 else 100 for i in g.nodes()], node_color = ['blue' if g.node[i]['Terrist Group'] == 1 else 'red' for i in g.nodes()])
    plt.title('Network connection')
    plt.xlabel('Time')
    plt.subplot(3, 2, 2)    # agent based model
    plt.cla()
    plt.imshow(Defence_level, cmap = plt.cm.OrRd,vmin = 0, vmax = 1)
    '''
    x = [ag.x for ag in popagents]
    y = [ag.y for ag in popagents]
    plot(y, x, 'b.') 
    '''
    x = [ a['Location'][0] for a in eventagents]
    y = [ a['Location'][1] for a in eventagents]
    plt.plot([a['Location'][1] for a in eventagents if a['Terrist Group']==1],[a['Location'][0] for a in eventagents if a['Terrist Group']==1],'r*')
    plt.plot([a['Location'][1] for a in eventagents if a['Terrist Group']==0],[a['Location'][0] for a in eventagents if a['Terrist Group']==0],'bo')
    for i in xrange(len(eventagents)):
        plt.annotate('Attack', (eventagents[i]['Location'][1],eventagents[i]['Location'][0]))

    plt.xlim(0, World_size_x)
    plt.ylim(0, World_size_y)
    #nx.draw(g, with_labels = True)
    
    plt.title('Locations of Iraq')
    plt.axis('off')
    plt.subplot(3, 2, 3)
    plt.cla()
    plt.plot(timet,agfnum1,'r-')
    plt.plot(timet,agfnum2,'b-')
    plt.title('Agressive Factors')
    plt.show()
    plt.subplot(3, 2, 4)
    plt.cla()
    plt.plot(timet,attnum1,'r-')
    plt.plot(timet,attnum2,'b-')
    plt.title('Number of Attacks')
    plt.show()
    plt.subplot(3, 2, 5)
    plt.cla()
    plt.plot(agfnum1,agfnum2)
    plt.title('Agressive Factors (Phase Space)')
    plt.show()
    
    plt.subplot(3, 2, 6)
    plt.cla()
    plt.plot(timet,defnum,'r-')
    plt.title('Mean of Defence Level')
    plt.show()


def update():
    global g,positions,World_size_x,World_size_y,popagents,timet, attnum1,attnum2,t,aggressivefactor1,aggressivefactor2
    global  Defence_level,eventagents, eventsize1, eventsize2,agfnum1,agfnum2,defnum
    olen=len(g.node)
    ##Generate an event
    G1kill=sum([g.node[i]['Killed People'] for i in g.nodes() if g.node[i]['Terrist Group']==0][-5:-1])
    G2kill=sum([g.node[i]['Killed People'] for i in g.nodes() if g.node[i]['Terrist Group']==1][-5:-1])
    
    # generate the number of new attacks
    eventsize1=int(aggressivefactor1*((len(popagents)/sum(Defence_level))+float(G1kill)/len(popagents)-np.log(sum(Defence_level)*10/defencefactor/World_size_x)))
    eventsize2=int(aggressivefactor2*((len(popagents)/sum(Defence_level))+float(G2kill)/len(popagents)-np.log(sum(Defence_level)*10/defencefactor/World_size_x)))
    # maximum attacks allowed in one time period
    if eventsize1 > int(np.mean(Defence_level)/limi):
        eventsize1  =int(np.mean(Defence_level)/limi)
    if eventsize2 > int(np.mean(Defence_level)/limi):
        eventsize2  = int(np.mean(Defence_level)/limi)        
    
    at1=0
    at2=0
    eventagents=[]
    
    # add new attacks
    for j in range(eventsize1+eventsize2):
        i=olen+j
        g.add_node(i)
        # give new attacks attributes based on the distribution
        rn=np.random.rand()
        ind=pb.find(At_cp>=rn)
        g.node[i]['Attack Type']=At[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Wt_cp>=rn)
        g.node[i]['Weapon Type']=Wt[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Tt_cp>=rn)
        g.node[i]['Target Type']=Tt[ind[0]]
        
        rn=np.random.rand()
        ind=pb.find(Pp_cp>=rn)
        g.node[i]['Property']=Pp[ind[0]]
                
        while True:
            Kp = int(gauss(Kp_mean,35.86))
            if Kp >= 0:
                break
            
        while True:
            Wp = int(gauss(Wp_mean,80.44))
            if Wp >= 0:
                break

        g.node[i]['Killed People'] = Kp
        g.node[i]['Wounded People'] = Wp
        
        # success rate decision tree
        if Kp <= 0:
            Succ = 0
        else:
            if g.node[i]['Attack Type'] != 'Assassination' and g.node[i]['Attack Type'] != 'Facility/Infrastructure Attack':
                Succ = 1
            elif g.node[i]['Attack Type'] == 'Facility/Infrastructure Attack':
                Succ = 0
            else:
                if Kp <=1:
                    Succ = 0
                else:
                    if g.node[i]['Target Type'] != 'Goverment (General)' and g.node[i]['Target Type'] != 'Government (Diplomatic)' and g.node[i]['Target Type'] != 'Private Citizens Property' and g.node[i]['Target Type'] != 'Military' and g.node[i]['Target Type'] != 'Police':
                        Succ = 1
                    elif g.node[i]['Target Type'] == 'Government (Diplomatic)':
                        Succ = 0
                    elif g.node[i]['Target Type'] == 'Goverment (General)':
                        if g.node[i]['Property'] == 'Unknown':
                            Succ = 1
                        else: 
                            Succ = 0
                    elif g.node[i]['Target Type'] == 'Military':
                        if g.node[i]['Property'] != 'Damage':
                            Succ = 1
                        else: 
                            Succ = 0
                    elif g.node[i]['Target Type'] == 'Private Citizens Property':
                        if Wp > 14 and Wp <= 38:
                            Succ = 0
                        else:
                            Succ = 1
                    elif g.node[i]['Target Type'] == 'Police':      
                        if g.node[i]['Property'] == 'Damage':
                            Succ = 1
                        elif g.node[i]['Property'] == 'No Damage':
                            Succ = 0                        
                        else:
                            if Wp > 10:
                                Succ = 0
                            else:
                                Succ = 1
        g.node[i]['Success Rate'] = Succ

        # random location
        g.node[i]['Location'] = (randint(0,World_size_x),randint(0,World_size_y))
        
        
        if j<eventsize1:
            g.node[i]['Terrist Group'] =0
            # define the attack will happen or not for terrorist group 0            
            attacksucess=g.node[i]['Success Rate']+float(G1kill)/len(popagents)  #len(popagents):population factor
            if  attacksucess>Defence_level[g.node[i]['Location'][0],g.node[i]['Location'][1]]/defencefactor:
                eventagents.append(g.node[i])
                aggressivefactor1+=aggressivechange1  # if an attark happened, the aggressive factor decrease
                at1+=1
            else:
               del g.node[i]
               aggressivefactor1+=aggressivechange2*(-2)  # if it failed to happen, the aggresive factor increase
        else:
            g.node[i]['Terrist Group'] =1
            # define the attack will happen or not for terrorist group 0   
            attacksucess=g.node[i]['Success Rate']+float(G2kill)/len(popagents)   #len(popagents):population factor
            if  attacksucess>Defence_level[g.node[i]['Location'][0],g.node[i]['Location'][1]]/defencefactor:
                eventagents.append(g.node[i])
                aggressivefactor2+=aggressivechange2
                at2+=1
            else:
               del g.node[i]
               aggressivefactor2+=aggressivechange2*(-2)
        
        
    # terrorist leader learn from the attacks history
        # reused one of the most common used attacks
    dc = g.degree()
    maxd = max(dc.values())
    hubs = [i for i in g.nodes_iter() if g.degree(i) == maxd]
    j = rd.choice(hubs)
    
    if np.random.rand()-np.mean(Defence_level)*Coef_meandefence > 0 and g.node[j]['Success Rate'] == 1:  
        ideg = len(g.node) 
        g.add_node(ideg)
        g.node[ideg]['Attack Type'] = g.node[j]['Attack Type']     # copy attack attributes
        g.node[ideg]['Weapon Type'] = g.node[j]['Weapon Type']
        g.node[ideg]['Target Type'] = g.node[j]['Target Type']
        g.node[ideg]['Property'] = g.node[j]['Property']
        # random location
        g.node[ideg]['Location'] = (randint(0,World_size_x),randint(0,World_size_y))
        
        if np.random.rand()<infoshare:     # the attack are more likely be learned by the group who made it than other groups
            g.node[ideg]['Terrist Group'] = g.node[j]['Terrist Group']
        elif g.node[ideg]['Terrist Group'] == 0:
            g.node[ideg]['Terrist Group'] = 1
        else:
            g.node[ideg]['Terrist Group'] = 0
        
        # assign number of killed and wounded
        while True:
            g.node[ideg]['Killed People'] = int(gauss(g.node[j]['Killed People'],g.node[j]['Killed People']))
            if g.node[ideg]['Killed People'] >= 0:
                break
                
        while True:
            g.node[ideg]['Wounded People'] = int(gauss(g.node[j]['Wounded People'],g.node[j]['Wounded People']))
            if g.node[ideg]['Wounded People'] >= 0:
                break
            
        
        g.node[ideg]['Success Rate'] = g.node[j]['Success Rate']

        if g.node[ideg]['Terrist Group'] ==0:
            attacksucess=g.node[ideg]['Success Rate']+float(G1kill)/len(popagents)
            if  attacksucess>Defence_level[g.node[ideg]['Location'][0],g.node[ideg]['Location'][1]]/defencefactor:
                eventagents.append(g.node[ideg])
                aggressivefactor1+=aggressivechange1
                at1+=1
            else:
               del g.node[ideg]
               aggressivefactor1+=aggressivechange2*(-2)
        else:
            attacksucess=g.node[ideg]['Success Rate']+float(G2kill)/len(popagents)
            if  attacksucess>Defence_level[g.node[ideg]['Location'][0],g.node[ideg]['Location'][1]]/defencefactor:
                eventagents.append(g.node[ideg])
                aggressivefactor2+=aggressivechange2
                at2+=1
            else:
               del g.node[ideg]
               aggressivefactor2+=aggressivechange2*(-2)
           
    # calculate the similarity for new attacks, build new edges
    if len(g.node)-olen>0:
        for i in xrange(len(g.node)):
            for j in range(i+1,len(g.node)):
                S_At = 0
                S_Wt = 0
                S_Tt = 0
                S_Kp = 0
                S_Wp = 0
                S_Pp = 0
                if g.node[i]['Attack Type'] == g.node[j]['Attack Type']:
                    S_At = 1
                if g.node[i]['Weapon Type'] == g.node[j]['Weapon Type']:
                    S_Wt = 1
                if g.node[i]['Target Type'] == g.node[j]['Target Type']:
                    S_Tt = 1
                if abs(g.node[i]['Killed People'] - g.node[j]['Killed People']) <= 0.2 * Kp_mean:
                    S_Kp = 1
                if abs(g.node[i]['Wounded People'] - g.node[j]['Wounded People']) <= 0.2 * Wp_mean:
                    S_Wp = 1
                if g.node[i]['Property'] == g.node[j]['Property']:
                    S_Pp = 1
                    # similarity function
                Simi = Weight * S_At + Weight * S_Wt + Weight * S_Tt + Weight * S_Kp + Weight * S_Wp + Weight * S_Pp+exp(-sqrt((g.node[j]['Location'][0]-g.node[i]['Location'][0])**2+ (g.node[i]['Location'][1]-g.node[i]['Location'][1])**2))*(Defence_level[g.node[i]['Location'][0],g.node[i]['Location'][1]]+Defence_level[g.node[j]['Location'][0],g.node[j]['Location'][1]])
           
           # simicon: similarity threshold
                if Simi >= simicons:
                    g.add_edge(i, j)
                
                # change defence level based on attacks
        for j in range(len(g.node)-olen):
            i=olen+j
            for x in range(World_size_x):
                for y in range(World_size_y):
                    if (x-g.node[i]['Location'][0])**2+ (y-g.node[i]['Location'][1])**2<DNeibor_size**2:
                        Defence_level[x,y]+=defencefactor*np.exp(-np.sqrt((x-g.node[i]['Location'][0])**2+ (y-g.node[i]['Location'][1])**2))
    

    

    '''   
    ## update population movement
    for ag in popagents:
        newx, newy = (ag.x + randint(-20, 20)) % World_size_x, (ag.y + randint(-20, 20)) % World_size_y
        diff = (Defence_level[newx, newy] -  Defence_level[ag.x, ag.y]) / defencefactor
        if random() < exp(diff) / (1 + exp(diff)):
            ag.x, ag.y = newx, newy
    '''
    
    ## defence cool down
    Defence_level-=dcooldown
    Defence_level[Defence_level<0]=0
    timet+=[t,]
    t+=1
    attnum1+=[at1,]   
    attnum2+=[at2,]  
    agfnum1+=[aggressivefactor1,]
    agfnum2+=[aggressivefactor2,]
    defnum += [np.mean(Defence_level),]
    
    # aggressive factor changes based on number of attacks made by the terrorist group. and the difference compare with other groups
    # at1: number of attacks of group 1 at time T, at2 number of attacks of group 2 at time T
    aggressivefactor1 += -aggressivefade+ 0.02/(at1+0.1) - 0.5*(at1-at2)
    aggressivefactor2 += -aggressivefade+ 0.02/(at2+0.1) - 0.5*(at2-at1)
    
    pass   
    
    ## attack event fade
    #g.remove_node(g.node[1])
    



In [3]:
#Run the simulator code for seen dynamic behaviors of terrorist interactions
import pycxsimulator
pycxsimulator.GUI().start(func=[initialize, observe, update])

AttributeError: module 'pylab' has no attribute 'find'